In [1]:
from datetime import datetime
from creodias_finder import query, download
import os
import fiona
import folium
from shapely.geometry import shape, mapping, Polygon
import json

In [2]:
def load_credentials():
    with open('credentials.json', 'r') as file:
        credentials = json.load(file)
    return credentials

In [3]:
def read_shapefile(shp_path):
    '''
    Reads a shapefile and extracts Shapely geometries from its features.

    Parameters:
    - shp_path (str): The path to the shapefile.

    Returns:
    - geometries (list): A list containing Shapely geometries.
    '''
    geometries = []  # Initialize an empty list to collect geometries
    
    # Open the shapefile
    with fiona.open(shp_path) as shp:
        # Iterate over features in the shapefile
        for feature in shp:
            # Extract the Shapely geometry from the feature and add it to the list
            geometry = shape(feature['geometry'])
            geometries.append(geometry)
    
    return geometries  # Return the list of geometries after the loop

In [4]:
def get_product_aoi(product, results):
    '''
    Extracts a Shapely Polygon geometry from a GeoJSON dictionary in the results.

    Parameters:
    - product (str): The product identifier.
    - results (dict): A dictionary containing results with GeoJSON geometries.

    Returns:
    - polygon (Polygon or None): A Shapely Polygon object if the GeoJSON type is 'Polygon', else None.
    '''
    geojson_dict = results.get(product).get('geometry')
    if geojson_dict['type'] == 'Polygon':
        # Extracting coordinates from the dictionary
        coordinates = geojson_dict['coordinates'][0]

        # Creating a Shapely Polygon object
        polygon = Polygon(coordinates)

        return polygon

In [5]:
def generate_random_color():
    '''
    Generates a random color in hexadecimal format (RGB).

    Returns:
    - color_hex (str): Random color code in the format "#RRGGBB" where RR, GG, and BB are hexadecimal values for red, green, and blue components.
    '''
    import random
    # Generate random RGB values
    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)

    # Convert RGB to hexadecimal format
    color_hex = "#{:02x}{:02x}{:02x}".format(red, green, blue)

    return color_hex

In [6]:
def visualize_geometries_folium(geometries, colors, popup_texts, zoom_start=None):
    '''
    Create an interactive Folium map to visualize Shapely geometries with customized colors and popups.

    Parameters:
    - geometries (list): A list of Shapely geometries to be visualized on the map.
    - colors (list): A list of colors corresponding to each geometry.
    - popup_texts (list): A list of strings containing popup text for each geometry.
    - zoom_start (int, optional): Initial zoom level for the map. If not provided, the zoom level is
      automatically determined based on the bounds of the first geometry.

    Returns:
    - m (folium.Map): Folium map object displaying the visualized geometries.
    '''
    
    # Extract bounds of the first geometry
    first_geometry_bounds = geometries[0].bounds

    # Use the bounds to set the initial zoom_start
    if zoom_start is None:
        # Calculate the initial zoom level based on the bounds of the first geometry
        delta_lat = first_geometry_bounds[3] - first_geometry_bounds[1]
        delta_lon = first_geometry_bounds[2] - first_geometry_bounds[0]

        zoom_start = 10  # You can adjust the default zoom level as needed

        if delta_lat > delta_lon:
            zoom_start = int(-1.3 * delta_lat + 20)
        else:
            zoom_start = int(-1.3 * delta_lon + 20)

    # Create a map centered at the first geometry
    m = folium.Map(location=[(first_geometry_bounds[1] + first_geometry_bounds[3]) / 2,
                             (first_geometry_bounds[0] + first_geometry_bounds[2]) / 2],
                   zoom_start=zoom_start)

    for geometry, color, popup_text in zip(geometries, colors, popup_texts):
        # Convert Shapely geometry to GeoJSON format
        geojson_feature = {
            "type": "Feature",
            "geometry": mapping(geometry),
            "properties": {"popup_text": popup_text}
        }

        # Create a FeatureGroup for each GeoJSON layer with its name as the popup_text
        geojson_group = folium.FeatureGroup(name=popup_text, show=True)

        # Add the GeoJSON feature to the GeoJSON group with the specified color
        geojson_layer = folium.GeoJson(
            geojson_feature,
            name='geojson',
            style_function=lambda x, color=color: {'fillColor': color, 'color': color},
        )
        
        # Bind a popup to the GeoJSON layer
        popup_html = f'Product ID: <b>{popup_text}</b>'
        popup = folium.Popup(popup_html, max_width=300)
        popup.add_to(geojson_layer)
        
        geojson_layer.add_to(geojson_group)

        # Add the GeoJSON group to the map
        geojson_group.add_to(m)

    # Add a LayerControl to toggle the visibility of each GeoJSON group
    folium.LayerControl().add_to(m)

    # Fit the map to the bounds of all geometries
    bounds = [geometry.bounds for geometry in geometries]
    min_lat, min_lon = min(bound[0] for bound in bounds), min(bound[1] for bound in bounds)
    max_lat, max_lon = max(bound[2] for bound in bounds), max(bound[3] for bound in bounds)
    m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

    return m

In [27]:
shp_path = '.../path-to-shapefile.shp' #
aoi_poly = read_shapefile(shp_path)

In [31]:
#Search for the Sentinel 5p products
start = datetime(2020, 1, 1) ##Year, month, day
end = datetime(2020, 1, 2)

results = query.query(
    'Sentinel5P',
    start_date=start,
    end_date=end,
    geometry=aoi_poly[0],
    status="ALL",
    productType = "L2__SO2___" #(L1B_IR_SIR, L1B_IR_UVN, L1B_RA_BD1, L1B_RA_BD2, L1B_RA_BD3, L1B_RA_BD4, L1B_RA_BD5, L1B_RA_BD6, L1B_RA_BD7, L1B_RA_BD8, L2__AER_AI, L2__AER_LH, L2__CH4___, L2__CLOUD_, L2__CO____, L2__HCHO__, L2__NO2___, L2__NP_BD3, L2__NP_BD6, L2__NP_BD7, L2__O3_TCL, L2__O3____, L2__SO2___)
)

polygons_geo = []
products_ids = []
for m in range(len(list(results.keys()))):
    product = list(results.keys())[m]
    polygons_geo.append(get_product_aoi(product, results))
    products_ids.append(product)

polygons_geo, products_ids = zip(*[(poly, pid) for poly, pid in zip(polygons_geo, products_ids) if poly is not None])
polygons_geo, products_ids = list(polygons_geo), list(products_ids)

# polygons_geo = list(filter(lambda x: x is not None, polygons_geo))
polygons_geo.insert(0, aoi_poly[0])
products_ids.insert(0, 'aoi_polygon')

In [33]:
# Specify colors for each polygon
colors = [generate_random_color() for _ in range(len(polygons_geo))]
popup_texts = products_ids.copy()

# Visualize the polygons on the same map
map_with_polygons = visualize_geometries_folium(polygons_geo, colors, popup_texts, zoom_start=1)
map_with_polygons

In [34]:
# ids = [result['id'] for result in results.values()]
ids = '998d8cf1-95cb-439a-8144-3b6d9f276be5'
output_path = '.../path-to-output'
credentials = load_credentials()

CREDENTIALS = {
    "username": credentials['user2'],
    "password": credentials['password2']
}

# download single product by product ID
download.download(ids, outfile=os.path.join(output_path, 'productsfile.zip'), **CREDENTIALS)

200


1.06GB [06:16, 2.83MB/s]


In [ ]:
# download a list of products, multithreaded
# download.download_list(ids[1:11], threads=10, **CREDENTIALS, outdir='/home/antonkout/Documents/modules/download_sentinel_5p/products')